### Actividad: Análisis "La Mañanera"
<p><kbd>0226594</kbd> Sara Carolina Gómez Delgado  </p>
Introducción a Python y análisis del lenguaje natural.

<kbd>https://www.youtube.com/watch?v=FILPdMMnYuA&ab_channel=Adri%C3%A1n</kbd>


In [30]:
import urllib.request
import requests
import glob
import re
import io
import os
from bs4 import BeautifulSoup
from datetime import datetime

### Read HTML code form URL

In [31]:
def getHTML(url, path):
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'})
    with open(path, "w") as f:
        f.write(response.text)
getHTML("https://presidente.gob.mx/secciones/version-estenografica/page/2/", "../data/Política/html-code.txt")

In [32]:
regex = '(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'
re.findall(regex, open("../data/Política/html-code.txt", "r", encoding = "utf-8").read())[:3]

[('http',
  '151.394.mwp.accessdomain.com',
  '/wp-content/themes/diplomat/favicon.ico'),
 ('https', 'presidente.gob.mx', '/feed/'),
 ('https', 'presidente.gob.mx', '/xmlrpc.php')]

###  Fetch each page

In [33]:
url_pagina_i = "https://presidente.gob.mx/secciones/version-estenografica/page/"
num_paginas = 117 # versiones estenográficas
for i in range(num_paginas): 
    getHTML(url_pagina_i + str(i + 1) + "/", "./paginas/" + str(i + 1) + ".txt")

In [34]:
urls_conf = []
for f_pagina in glob.glob("./paginas/*"):
    urls_conf += [path for _, _, path in re.findall(regex, open(f_pagina,"r", encoding="utf-8").read()) 
                if "estenografica-de-la" in path] # get urls with "estenografica-de-la"

In [35]:
urls_conf_set = list(set(urls_conf))
print(len(urls_conf_set))

1021


In [36]:
for path in urls_conf_set:
    getHTML("https://presidente.gob.mx" + path, "./estenograficas/" + path.replace("/", "-"))

###  Clean data (only text)

In [37]:
for f_pagina in glob.glob("./estenograficas/*"):
    soup = BeautifulSoup(open(f_pagina, "r", encoding = "utf-8").read(), 'html.parser')
    f_pagina = f_pagina.replace("\\", "")
    newPath = "./estenograficas_limpias/" + f_pagina.replace("./estenograficas", "")
    open(newPath, "w", encoding = "utf-8").write(soup.get_text())

### Normalizar cada nombre

In [39]:
def normaliza(str_fecha):
    datetimeobject = datetime.strptime(str_fecha[:6] + "20" + str_fecha[6:], '%d.%m.%Y') 
    newformat = datetimeobject.strftime('%Y-%m-%d')
    return newformat
    

i = 0
fechas_conferencia = []
for f_pagina in glob.glob("./estenograficas_limpias/*"):
    texto = open(f_pagina, 'r', encoding = 'utf-8').read()
    fechas_en_archivo = re.findall('[0-9][0-9]\.[0-9][0-9]\.[0-9][0-9]', texto)
    if len(fechas_en_archivo) == 4:
        path = './estenograficas_limpias_por_fecha/' + normaliza(fechas_en_archivo[0])
        if os.path.isfile(path): 
            if not os.path.isfile(path + "_2da"):
                path += "_2da"
                print("---> Segunda:", f_pagina)
            elif not os.path.isfile(path + "_3ra"):
                path += "_3ra"
                print("--->Tercera:", f_pagina)
        open(path, 'w', encoding = "utf-8").write(texto)
        fechas_conferencia += [normaliza(fechas_en_archivo[0])]
    else:
        print(":S Raro:", f_pagina)

:S Raro: ./estenograficas_limpias\-08-1020-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
---> Segunda: ./estenograficas_limpias\-12-11-21-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-desde-sonora-
---> Segunda: ./estenograficas_limpias\-13-05-2022-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-desde-nuevo-leon-
---> Segunda: ./estenograficas_limpias\-29-05-19-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-2-
:S Raro: ./estenograficas_limpias\-29-0920-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador-
---> Segunda: ./estenograficas_limpias\-version-estenografica-de-la-conferencia-de-prensa-extraordinaria-del-presidente-andres-manuel-lopez-obrador-2-
--->Tercera: ./estenograficas_limpias\-version-estenografica-de-la-